In [ ]:
# Importing the required packages
import os
import re
import math
import uuid
import datetime as dt
import pandas as pd
import numpy as np
import lxml.html as lh

from lxml import etree as et
from lxml.builder import E

In [8]:
# Setting the current working directory
path = 'C:\Users\HP\Desktop\BIM Coding\Navisworks_XML'
os.chdir(path)
os.getcwd()

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<ipython-input-8-47c32c9c0789>, line 2)

In [78]:
# The code below generates a string format of the current date and time.
now = dt.datetime.now().strftime("%Y-%m-%d") 

# Set .xml file name below. Change default name "PK_Clash_Tests_Infra" as required.
# This is the .xml file that will be generated after all the code runs successfully.
FILE_NAME = f"{now}-Navisworks_Clash_full"

In [79]:
# DataFrame from clash detection matrix
cdm = pd.read_excel(
    'Navisworks_Clash_Matrix_Checks.xlsx', 
    sheet_name=0, 
    index_col=[0,1,2,3], 
    header=[0,1,2,3]
)

cdm.head()

ARCHITECTURE                                                      ARCHITECTURE  \
Doors                                                                    Doors   
36" x 84"                                                            72" x 84"   
NaN                                                                        NaN   
NaN NaN   72" x 84"                                          H-50          NaN   
    Walls Exterior - Brick and CMU on MTL. Stud              H-50        H-200   
          Exterior - Brick and CMU on MTL. Stud (No Parapet) H-50       H-1000   
          Generic - 8" Masonry                               H-50        H-400   
          Interior - 6 1/8" Partition (2-hr)                 H-50        H-400   

ARCHITECTURE                                                                                             \
Doors                                                                                             Walls   
36" x 84"                                                         Exterior - Brick and CMU on MTL. Stud   
NaN                                                                                                 NaN   
NaN NaN   72" x 84"                                          H-50                                   NaN   
    Walls Exterior - Brick and CMU on MTL. Stud              H-50                                   NaN   
          Exterior - Brick and CMU on MTL. Stud (No Parapet) H-50                                  H-50   
          Generic - 8" Masonry                               H-50                                  H-50   
          Interior - 6 1/8" Partition (2-hr)                 H-50                                  H-50   

ARCHITECTURE                                                                                                          \
Doors                                                                                                                  
36" x 84"                                                         Exterior - Brick and CMU on MTL. Stud (No Parapet)   
NaN                                                                                                              NaN   
NaN NaN   72" x 84"                                          H-50                                                NaN   
    Walls Exterior - Brick and CMU on MTL. Stud              H-50                                                NaN   
          Exterior - Brick and CMU on MTL. Stud (No Parapet) H-50                                                NaN   
          Generic - 8" Masonry                               H-50                                               H-50   
          Interior - 6 1/8" Partition (2-hr)                 H-50                                               H-50   

ARCHITECTURE                                                                            \
Doors                                                                                    
36" x 84"                                                         Generic - 8" Masonry   
NaN                                                                                NaN   
NaN NaN   72" x 84"                                          H-50                  NaN   
    Walls Exterior - Brick and CMU on MTL. Stud              H-50                C-200   
          Exterior - Brick and CMU on MTL. Stud (No Parapet) H-50               C-1400   
          Generic - 8" Masonry                               H-50               C-1000   
          Interior - 6 1/8" Partition (2-hr)                 H-50               C-1000   

ARCHITECTURE                                                                                          \
Doors                                                                                                  
36" x 84"                                                         Interior - 6 1/8" Partition (2-hr)   
NaN                                                                                          

In [80]:
cdm.info()

# Number of rows
# len(cdm.index)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 17 entries, (nan, nan, '72" x 84"', 'H-50') to ('STRUCTURE', 'Structural Columns', 'HSS6X6X5/8', 'H-50')
Data columns (total 17 columns):
 #   Column                                                                          Non-Null Count  Dtype 
---  ------                                                                          --------------  ----- 
 0   (ARCHITECTURE, Doors, 72" x 84", NaN)                                           16 non-null     object
 1   (ARCHITECTURE, Walls, Exterior - Brick and CMU on MTL. Stud, NaN)               15 non-null     object
 2   (ARCHITECTURE, Walls, Exterior - Brick and CMU on MTL. Stud (No Parapet), NaN)  14 non-null     object
 3   (ARCHITECTURE, Walls, Generic - 8" Masonry, NaN)                                16 non-null     object
 4   (ARCHITECTURE, Walls, Interior - 6 1/8" Partition (2-hr), NaN)                  16 non-null     object
 5   (ARCHITECTURE, Walls, 18" Concrete, NaN)           

In [81]:
cdm_val = cdm.values.tolist()  # list to store all the clash detection matrix values (except index and header)
cdm_idx = cdm.index.to_list()  # list to store the clash detection matrix multi-index  

clash_tests_names = []  # clash test names
clash_tols_types = []  # clash tolerance and type (not split!)
rgt_loc = []  # right locator
lft_loc = []  # left locator

for idx_r, row in enumerate(cdm_val):
    for idx_c, clash_tol_type in enumerate(row):
        if (clash_tol_type is not np.nan) and (type(clash_tol_type) is str):
            clash_grp_c = cdm_idx[idx_c][2] # Clash group from column
            clash_grp_r = cdm_idx[idx_r][2] # Clash group from row
            clash_test = clash_grp_r + '_v_' + clash_grp_c
            # Left locator construction
            lft_con = f"lcop_selection_set_tree/{cdm_idx[idx_r][0]}/{cdm_idx[idx_r][1]}-{clash_grp_r}"
            # Right locator construction
            rgt_con = f"lcop_selection_set_tree/{cdm_idx[idx_c][0]}/{cdm_idx[idx_c][1]}-{clash_grp_c}"
            # Appending to lists
            clash_tests_names.append(clash_test)
            clash_tols_types.append(clash_tol_type)
            rgt_loc.append(rgt_con)
            lft_loc.append(lft_con)
            # print(f'Clash Group: {clash_grp_r}, Type & Tolerance: {clash_tols_types}, Clash Test: {clash_test}')
        else:
            pass

In [82]:
clash_types = []  # clash type
clash_tols = [(int(i[2:]) / 1000) for i in clash_tols_types]  # clash tolerance

for i in clash_tols_types:
    if i[0].lower() == 'c':
        clash_types.append("clearance")
    elif i[0].lower() == 'h':
        clash_types.append("hard")
    else:
        pass

In [83]:
# li = [row[???] for row in cdm.itertuples(index=False, name=None)]
# alternative approach can be explored using pandas.DataFrame.itertuples
# it would also include the use of enumerate(cdm.itertuples())

In [84]:
Element_category = [cdm_idx[idx][1] for idx, row in enumerate(cdm_idx)]  # Clash priority list
Element_type = [cdm_idx[idx][2] for idx, row in enumerate(cdm_idx)]  # Clash group list
disciplines = [cdm_idx[idx][0] for idx, row in enumerate(cdm_idx)]  # Discipline list

In [85]:
dfSS = pd.DataFrame({
    'Element Category': Element_category,
    'Element Type': Element_type,
    'Discipline': disciplines,
})

dfSS.head()

,Element Category,Element Type,Discipline
0,NaN,"72"" x 84""",NaN
1,Walls,Exterior - Brick and CMU on MTL. Stud,NaN
2,Walls,Exterior - Brick and CMU on MTL. Stud (No Para...,NaN
3,Walls,"Generic - 8"" Masonry",NaN
4,Walls,"Interior - 6 1/8"" Partition (2-hr)",NaN


In [86]:
# getting unique values for clash priority and discipline

cPrLi_unq = dfSS['Element Category'].unique().tolist()  # unique clash priority
dispLi_unq = dfSS['Discipline'].unique().tolist()  # unique clash discipline

In [87]:
# # Above below can be used for generating clash test combinations 
# import itertools as it
# y = [' vs '.join(val) for val in it.combinations(dispLi_unq, 2)]

In [88]:
# Setting the xml schema and generating the root

qnmAtt = et.QName("http://www.w3.org/2001/XMLSchema-instance", "noNamespaceSchemaLocation")

ct_root = et.Element(
    'exchange',
    {qnmAtt: "http://download.autodesk.com/us/navisworks/schemas/nw-exchange-12.0.xsd"},
    units="m"
)

baTest = et.Element(
    'batchtest', {
        "name": FILE_NAME, 
        "internal_name": FILE_NAME, 
        "units": "m"
        } 
)

ct_root.append(baTest)

In [9]:
clashtests = et.SubElement(baTest, "clashtests")

NameError: name 'BaTest' is not defined

In [90]:
# Generating clash test xml
for idx_cT, cT in enumerate(clash_tests_names):
    tT = clash_types[idx_cT]  # test type
    cTl = clash_tols[idx_cT]  # tolerance
    rLoc = rgt_loc[idx_cT]  # right locator
    lLoc = lft_loc[idx_cT]  # left locator
    clashtest = et.SubElement(
        clashtests, "clashtest", {
            "name": str(cT),
            "test_type": str(tT),
            "status": "new",
            "tolerance": str(cTl),
            "merge_composites": "0"
        }
    )

    linkage = et.SubElement(
        clashtest, "linkage", {
            "mode": "none"
        }
    )  

    left = et.SubElement(clashtest, "left")
    # left children
    clashselectionL = et.SubElement(
        left, "clashselection", {
            "selfintersect": "0",
            "primtypes": "1" 
        }
    )
    # clashselection child (left locator)
    lLocator = et.SubElement(clashselectionL, "locator")
    lLocator.text = lLoc1

    right = et.SubElement(clashtest, "right")  
    # right children
    clashselectionR = et.SubElement(
        right, "clashselection", {
            "selfintersect": "0",
            "primtypes": "1" 
        }
    )
    # clashselection child (right locator)
    rLocator = et.SubElement(clashselectionR, "locator")
    rLocator.text = rLoc

    rules = et.SubElement(clashtest, "rules")

In [91]:
# ct_root[0].remove(ct_root[0][0])  #removing elements

In [92]:
print(len(ct_root[0][0]))
# for e in ct_root[0][0]:
#     print(e.tag)

238


In [93]:
print(et.tostring(ct_root, pretty_print=True, encoding="UTF-8", xml_declaration=True).decode('utf-8'))

tree/nan/Walls-18" Concrete</locator>
          </clashselection>
        </right>
        <rules/>
      </clashtest>
      <clashtest name="W14X30_v_4 1/2” Soffit" test_type="hard" status="new" tolerance="0.05" merge_composites="0">
        <linkage mode="none"/>
        <left>
          <clashselection selfintersect="0" primtypes="1">
            <locator>lcop_selection_set_tree/STRUCTURE/Structural Framing-W14X30</locator>
          </clashselection>
        </left>
        <right>
          <clashselection selfintersect="0" primtypes="1">
            <locator>lcop_selection_set_tree/nan/Walls-4 1/2” Soffit</locator>
          </clashselection>
        </right>
        <rules/>
      </clashtest>
      <clashtest name="W14X30_v_84&quot; x 84&quot;" test_type="hard" status="new" tolerance="0.1" merge_composites="0">
        <linkage mode="none"/>
        <left>
          <clashselection selfintersect="0" primtypes="1">
            <locator>lcop_selection_set_tree/STRUCTURE/Structura

In [94]:
selectionsets = et.SubElement(baTest, 'selectionsets')

In [95]:
# Creating and appending all viewfolders based on discipline, to selectionsets
for disp in dispLi_unq:
    viewfolder = et.SubElement(
        selectionsets, 'viewfolder', {
            "name": str(disp),
            "guid": str(uuid.uuid4())
        }
    )

In [96]:
# Generating all selectionsets
for disp in dispLi_unq:
    vwFr = ct_root.find(f'.//viewfolder[@name="{disp}"]')
    for row in dfSS.itertuples(index=False, name=None):  # enumerate not required
        if row[2] == disp:
            # Constructing the selectionset name
            ssCon = [row[0], row[1]]
            ssName = '-'.join(ssCon)
            # Creating and appending selectionsets 
            selectionset = et.SubElement(
                vwFr, 'selectionset', {
                    "name": str(ssName),
                    "guid": str(uuid.uuid4())
                }
            )
            
            findspec = et.SubElement(
                selectionset, 'findspec', {
                    "mode": "all",
                    "disjoint": "0"
                }
            )

            conditions = et.SubElement(findspec, 'conditions')

            locator = et.SubElement(findspec, 'locator')
            locator.text = "/"

            # conditions children
            condition1 = et.SubElement(
                conditions, 'condition', {
                    "test": "contains",
                    "flags": "10"
                }
            )

            category1 = et.SubElement(condition1, 'category')

            nmCt1 = et.SubElement(
                category1, 'name', {
                    "internal": "AecDbPropertySet"
                }
            ) 
            nmCt1.text = "Element"

            prop1 = et.SubElement(condition1, 'property')

            nmPp1 = et.SubElement(
                prop1, 'name', {
                    "internal": "1"
                }
            ) 
            nmPp1.text = "Category"

            value1 = et.SubElement(condition1, 'value')

            data1 = et.SubElement(
                value1, "data", {
                    "type": "wstring"
                }
            )
            data1.text = str(row[0])

            # second condition
            condition2 = et.SubElement(
                conditions, 'condition', {
                    "test": "contains",
                    "flags": "10"
                }
            )

            category2 = et.SubElement(condition2, 'category')

            nmCt2 = et.SubElement(
                category2, 'name', {
                    "internal": "AecDbPropertySet"
                }
            ) 
            nmCt2.text = "Element"

            prop2 = et.SubElement(condition2, 'property')

            nmPp2 = et.SubElement(
                prop2, 'name', {
                    "internal": "3"
                }
            ) 
            nmPp2.text = "Type"

            value2 = et.SubElement(condition2, 'value')

            data2 = et.SubElement(
                value2, "data", {
                    "type": "wstring"
                }
            )
            data2.text = str(row[1])
        else:
            pass

In [97]:
# code below searches for the first viewfolder then shows its xml structure
vwFr_all = ct_root.find('.//viewfolder')
print(et.tostring(vwFr_all, pretty_print=True).decode('utf-8'))

<viewfolder xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" name="nan" guid="a791f0c2-a571-4186-a251-2092405d3aa9"/>



In [98]:
# # handy code for searching for, and deleting elements.
# # use elementnamexxx.attrib['nameofattrib'] = 'newnameofattrib' to change element attributes.
# # https://stackoverflow.com/questions/8171146/python-lxml-modify-attributes
# vfSch = ct_root.find('.//viewfolder[@name="SL"]')   # search based on attribute

# deLE = ct_root.find('.//testttt')

# deLE.getparent().remove(deLE)

# print(et.tostring(vfSch, pretty_print=True).decode('utf-8'))

In [99]:
ctData = et.tostring(ct_root, pretty_print=True, encoding="UTF-8", xml_declaration=True)

In [100]:
with open(f"{FILE_NAME}.xml", "wb") as nfXML:
    nfXML.write(ctData)